In [17]:
#IMPORTS
import shutil
import os
import pandas as pd
import numpy as np
import datetime
import itertools
import re
from dotenv import load_dotenv, dotenv_values
import json

In [18]:
#DOTENV
load_dotenv()
config = dotenv_values('.env')
config["WORKING_DIRECTORY"]

'D://THOR CUSTOMS BROKERAGE 3'

In [19]:
#IMPORTANT CONSTANTS
YEARS = ["2025", "2024", "2023", "2022", "2021", "2020"]
MONTHS = [
    {"month": "January", "number": "01"}, 
    {"month": "February", "number": "02"}, 
    {"month": "March", "number": "03"}, 
    {"month": "April", "number": "04"}, 
    {"month": "May", "number": "05"}, 
    {"month": "June", "number": "06"}, 
    {"month": "July", "number": "07"}, 
    {"month": "August", "number": "08"}, 
    {"month": "September", "number": "09"}, 
    {"month": "October", "number": "10"}, 
    {"month": "November", "number": "11"}, 
    {"month": "December", "number": "12"}
]

In [ ]:
#CREATE THE FILE STRUCTURE
def createSolutionStructure():

    solutionFolders = [config["EXCEL_SOLUTION_PATH"], config["PDF_SOLUTION_PATH"]]

    if not os.path.exists(config["SOLUTION_PATH"]):
        os.mkdir(config["SOLUTION_PATH"])
        os.mkdir(config["EXCEL_SOLUTION_PATH"])
        os.mkdir(config["PDF_SOLUTION_PATH"])
        for solutionFolder in solutionFolders:
            print(solutionFolder)
            for year in YEARS:
                YEAR_PATH = f"{solutionFolder + "/" + str(year)}"
                os.mkdir(YEAR_PATH)
                for month in MONTHS:
                    MONTH_PATH = f"{YEAR_PATH + "/" + month["number"] + "-" + month["month"]}"
                    os.mkdir(MONTH_PATH)
            if "PDF" in solutionFolder:
                os.mkdir(f"{config["PDF_SOLUTION_PATH"] + "/" + "Unknown"}")

createSolutionStructure()

In [27]:
def createBacklogFile(backlog):
    with open('backlogs.json', 'w', encoding='utf-8') as f:
        json.dump(backlog, f, ensure_ascii=False, indent=4)
    print(backlog)

In [ ]:
#RENAME EXCEL FILES AND ADD THESE FILES TO THEIR RESPECTIVE FOLDERS
monthCounts = {
    "January": 0, 
    "February": 1, 
    "March": 2, 
    "April": 3, 
    "May": 4, 
    "June": 5, 
    "July": 6, 
    "August": 7, 
    "September": 8, 
    "October": 9, 
    "November": 10, 
    "December": 11
}

excelFiles = os.listdir(config["EXCEL_FILE_PATH"])

def processExcelFiles():
    jsonFile = {
        "billingStatements": []
    }

    for excelFile in excelFiles:


        df = pd.read_excel(config["EXCEL_FILE_PATH"] + "/" + excelFile).replace("NAN", "1")
        containers = []
        month = year = None
        path = config["EXCEL_FILE_PATH"] + "/" + excelFile
        soaNumber = ""
        prevCell = ""

        for col in df:
            for cell in df[col]:

                if prevCell == "SOA":
                    soaNumber = str(cell)
                    prevCell = ""

                if pd.isna(cell) or isinstance(cell, datetime.datetime) or isinstance(cell, int) or isinstance(cell, float):
                    continue

                if ((len(cell) == 11 and cell.isalnum()) or (len(cell) == 13 and "-" in cell) or (len(cell) == 12 and "-" in cell) or "STRIPPING" in cell):

                    container1 = cell
                    if "-" in container1:
                        container1 = container1.replace("-", "")

                    containers.append(container1)
                
                if "date:" in cell.lower():
                    dateString = re.findall(r'[a-zA-Z]+|\d{4}', cell)
                    month = dateString[1].capitalize()

                    if month == "Febuary":
                        month = "February"

                    try:
                        year = dateString[2]
                    except:
                        year = "2023"

                    if month == "Paril":
                        month = "April"
                    if year == "220":
                        year = "2020"
                    if year == "221":
                        year = "2021"
                    if year == "222":
                        year = "2022"
                    if year == "223":
                        year = "2023"
                    if year == "224":
                        year = "2024"
                    if year == "225":
                        year = "2025"

                if cell == "SOA":
                    prevCell = "SOA"

        if soaNumber == "nan":
            soaNumber = "637"

        excelFilename = excelFile.split(".")[0]
        excelFilenameCopy = excelFilename
        bsNumber = excelFilenameCopy.split(" ")[0]
        
        excelFilename += " "
        excelFilename += str(soaNumber)
        
        excelFilename += ".xlsx"

        information = dict(
            fileName = excelFile, 
            bsNumber = bsNumber,
            soaNumber = soaNumber,
            year = year, 
            month = month, 
            containers = containers, 
            excelPath = ["STRUCTURED", "FILES", year, MONTHS[monthCounts[month]]["number"] + "-" + month, bsNumber],
            associatedPdfs = []
        )

        jsonFile["billingStatements"].append(information)

        bsNumberPath = f"{config["EXCEL_SOLUTION_PATH"] + "/" + year + "/" + MONTHS[monthCounts[month]]["number"] + "-" + month + "/" + bsNumber}"

        if not os.path.isdir(bsNumberPath):
            os.mkdir(bsNumberPath)
        shutil.copy(config["EXCEL_FILE_PATH"] + "/" + excelFile, config["EXCEL_SOLUTION_PATH"] + "/" + year +  "/" + MONTHS[monthCounts[month]]["number"] + "-" + month + "/" + bsNumber + "/" + excelFilename)

    return jsonFile

jsonData = processExcelFiles()

In [ ]:
createBacklogFile(jsonData)

In [29]:
def find(lst, key, value):
    for i, dic in enumerate(lst):
        if value == dic[key]:
            return i
    return None

In [ ]:
#ADD ASSOCIATED PDFS
pdfFiles = os.listdir(config["PDF_FILE_PATH"])
pdfFilesCopy = os.listdir(config["PDF_FILE_PATH"])
pdfFilesWithMatch = []

numberOfPdfsAttached = 0
xlsxExtensionLength = 5
pdfExtensionLength = 4

print(pdfFiles)
for year in YEARS:
    for month in MONTHS:
        folders = os.listdir(config["EXCEL_SOLUTION_PATH"] + "/" + year + "/" + month["number"] + "-" + month["month"])
        for folder in folders:
            excelFiles = os.listdir(config["EXCEL_SOLUTION_PATH"] + "/" + year + "/" + month["number"] + "-" + month["month"] + "/" + folder)
            for excelFile in excelFiles:

                matchingPdfs = [] 

                excelFilename = excelFile[:-xlsxExtensionLength]

                excelParts = excelFilename.split(" ")

                excelBSNumber = excelParts[0]
                excelSOANumber = excelParts[-1]

                specialCase = False

                for pdfFile in pdfFiles:

                    pdfFilename = pdfFile[:-pdfExtensionLength]
                    pdfParts = pdfFilename.split(" ")

                    pdfBSNumber = pdfParts[0]
                    pdfSOANumber = pdfParts[-1]

                    if pdfBSNumber == excelBSNumber:
                        matchingPdfs.append(pdfFile)
                        pdfFilesWithMatch.append(pdfFile)
                        numberOfPdfsAttached = numberOfPdfsAttached + 1

                if len(matchingPdfs) > 0:
                    ndx = find(jsonData["billingStatements"], "bsNumber", excelBSNumber)
                    if ndx is not None:
                        jsonData["billingStatements"][ndx]["associatedPdfs"] = matchingPdfs.copy()
                    for matchingPdf in matchingPdfs:
                        print(matchingPdf)
                        shutil.copy(config["PDF_FILE_PATH"] + "/" + matchingPdf, config["EXCEL_SOLUTION_PATH"] + "/" + year + "/" + month["number"] + "-" + month["month"] + "/" + folder + "/" + matchingPdf)
                        
pdfFilesNoMatch = [pdf for pdf in pdfFilesCopy if pdf not in pdfFilesWithMatch]

In [ ]:
createBacklogFile(jsonData)

In [32]:
print(pdfFilesNoMatch)

[]


In [33]:
print(numberOfPdfsAttached)

2284


In [34]:
pdfFiles = os.listdir(config["PDF_FILE_PATH"])
pdfFilesCopy = os.listdir(config["PDF_FILE_PATH"])
pdfFilesWithMatch = []

numberOfPdfsAttached = 0

for year in YEARS:
    for month in MONTHS:
        excelFiles = os.listdir(config["EXCEL_SOLUTION_PATH"] + "/" + year + "/" + month["number"] + "-" + month["month"])
        for excelFile in excelFiles:

            excelFileParts = excelFile.split()
            excelFilePartOne = excelFile.split(".")[0]

            newParts = excelFile.split(".")

            excelSoaNumber = excelFile.split()[-1].replace('.xlsx', '')

            code = excelFileParts[0]
            originalCode = excelFileParts[0]

            if "MSTSC" in code:
                code = code.replace("MSTSCIN", "MSTCIN")

            matchingPdfs = [] 

            for pdfFile in pdfFiles:
                parts = pdfFile.split(".")[0].split(" ")

                exists = False
                for part in parts:
                    if excelSoaNumber == part:
                        exists = True
                        break
                if exists == True:
                    matchingPdfs.append(pdfFile)
                    pdfFilesWithMatch.append(pdfFile)
                    numberOfPdfsAttached = numberOfPdfsAttached + 1

            excelName = excelFile.replace(".xlsx", "").split()

            if len(matchingPdfs) > 0:
                ndx = find(jsonData["billingStatements"], "soaNumber", excelSoaNumber)
                if ndx is not None:
                    jsonData["billingStatements"][ndx]["associatedPdfs"] = matchingPdfs.copy()

            for matchingPdf in matchingPdfs:
                pdfFilename = matchingPdf.split(".")[0] + ".pdf"
                shutil.copy(config["PDF_FILE_PATH"] + "/" + matchingPdf, config["PDF_SOLUTION_PATH"] + "/" + year + "/" + month["number"] + "-" + month["month"] + "/" + pdfFilename)

            pdfFiles = [pdf for pdf in pdfFiles if pdf not in matchingPdfs]

pdfFilesNoMatch = [pdf for pdf in pdfFilesCopy if pdf not in pdfFilesWithMatch]

for pdfFile in pdfFilesNoMatch:
    shutil.copy(config["PDF_FILE_PATH"] + "/" + pdfFile, config["PDF_SOLUTION_PATH"] + "/" + "Unknown" + "/" + pdfFile)

In [ ]:
createBacklogFile(jsonData)

In [ ]:
numberOfPdfsAttached
#######################################################

851

In [ ]:
with open('backlogs.json', 'w', encoding='utf-8') as f:
    json.dump(jsonData, f, ensure_ascii=False, indent=4)
print(jsonData)